<a href="https://colab.research.google.com/github/TurkuNLP/Turku-neural-parser-pipeline/blob/master/docs/tnpp_diaparse.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Turku Neural Parser Pipeline

* A mini-tutorial of the latest version of the parser pipeline
* Make sure to run it with GPU enabled (Runtime -> Change runtime type -> GPU)


# INSTALL

* git clone the code
* cd to the directory
* and install all requirements
* this does take its time, the parser leans on quite large libraries

In [1]:
!git clone https://github.com/TurkuNLP/Turku-neural-parser-pipeline.git
%cd Turku-neural-parser-pipeline

Cloning into 'Turku-neural-parser-pipeline'...
remote: Enumerating objects: 1140, done.
remote: Counting objects: 100% (181/181), done.
remote: Compressing objects: 100% (138/138), done.
remote: Total 1140 (delta 98), reused 95 (delta 43), pack-reused 959
Receiving objects: 100% (1140/1140), 328.94 KiB | 4.16 MiB/s, done.
Resolving deltas: 100% (656/656), done.
/content/Turku-neural-parser-pipeline


In [2]:
#I like to upgrade these first
!python3 -m pip install --upgrade pip
!python3 -m pip install --upgrade setuptools

     |████████████████████████████████| 1.6 MB 5.3 MB/s 
  Attempting uninstall: pip
    Found existing installation: pip 21.1.3
    Uninstalling pip-21.1.3:
      Successfully uninstalled pip-21.1.3


In [3]:
!python3 -m pip install -r requirements.txt

     |████████████████████████████████| 304 kB 5.3 MB/s 
     |████████████████████████████████| 2.6 MB 47.2 MB/s 
     |████████████████████████████████| 212 kB 53.8 MB/s 
     |████████████████████████████████| 69 kB 5.2 MB/s 
     |████████████████████████████████| 918 kB 54.5 MB/s 
     |████████████████████████████████| 272 kB 74.2 MB/s 
     |████████████████████████████████| 22.3 MB 1.2 MB/s 
     |████████████████████████████████| 895 kB 52.6 MB/s 
     |████████████████████████████████| 3.3 MB 56.9 MB/s 
     |████████████████████████████████| 636 kB 33.7 MB/s 
     |████████████████████████████████| 212 kB 42.8 MB/s 
     |████████████████████████████████| 209 kB 60.2 MB/s 
     |████████████████████████████████| 207 kB 55.0 MB/s 
     |████████████████████████████████| 206 kB 60.2 MB/s 
     |████████████████████████████████| 195 kB 58.5 MB/s 
     |████████████████████████████████| 53 kB 1.3 MB/s 
     |████████████████████████████████| 56 kB 3.7 MB/s 
     |███████████████

In [4]:
!python3 -m pip install --no-deps OpenNMT-py==2.1.2 

  Using cached OpenNMT_py-2.1.2-py3-none-any.whl (212 kB)
  Attempting uninstall: OpenNMT-py
    Found existing installation: OpenNMT-py 1.2.0
    Uninstalling OpenNMT-py-1.2.0:
      Successfully uninstalled OpenNMT-py-1.2.0


# FETCH MODEL

* At present, only the Finnish (fi_tdt_dia) and English (en_ewt_dia) models are available for the most recent diaparser-based version of the pipeline
* ...more to come...

In [5]:
!python3 fetch_models.py en_ewt_dia

* The above command created the directory `models_fi_tdt_dia` with the model
* The file `models_fi_tdt_dia/pipelines.yaml` defines all the possible pipelines for the parser in this model
* Currently, only the `parse_plaintext` one works

# PARSE IN PYTHON

* You need to load and start the pipeline of choice
* Like so:

In [6]:
from tnparser.pipeline import read_pipelines, Pipeline

text1="Minulla on söpö koira. Se haukkuu, syö makkaraa,\
  jahtaa oravia ja tsillailee kanssani!"


# What pipelines do we have for the Finnish model?
available_pipelines=read_pipelines("models_en_ewt_dia/pipelines.yaml")               # {pipeline_name -> its steps}
# This is a dictionary, its keys are the pipelines
print(list(available_pipelines.keys()))
# Instantiate one of the pipelines
p=Pipeline(available_pipelines["parse_plaintext"])    

['parse_plaintext', 'tag_plaintext', 'parse_sentlines', 'parse_wslines', 'parse_conllu', 'tokenize']


/usr/local/lib/python3.7/dist-packages/pytorch_lightning/core/decorators.py:66: LightningDeprecationWarning: The `@auto_move_data` decorator is deprecated in v1.3 and will be removed in v1.5. Please use `trainer.predict` instead for inference. The decorator was applied to `predict`
  "The `@auto_move_data` decorator is deprecated in v1.3 and will be removed in v1.5."
INFO:root:Loading model from /content/Turku-neural-parser-pipeline/models_en_ewt_dia/Tagger/best.ckpt


In [7]:
print(p.parse(text1))

# newdoc
# newpar
# sent_id = 1
# text = Minulla on söpö koira.
1	Minulla	Minulla	PROPN	_	Number=Sing	0	root	_	_
2	on	on	ADP	_	_	4	case	_	_
3	söpö	söpö	PROPN	_	Number=Sing	4	compound	_	_
4	koira	koira	NOUN	_	Number=Sing	1	nmod	_	SpaceAfter=No
5	.	.	PUNCT	_	_	1	punct	_	_

# sent_id = 2
# text = Se haukkuu, syö makkaraa, jahtaa oravia ja tsillailee kanssani!
1	Se	Se	PROPN	_	Number=Sing	2	discourse	_	_
2	haukkuu	haukkuu	PROPN	_	Number=Sing	0	root	_	SpaceAfter=No
3	,	,	PUNCT	_	_	5	punct	_	_
4	syö	sy'	PROPN	_	_	5	compound	_	_
5	makkaraa	makkaraa	PROPN	_	Number=Sing	2	vocative	_	SpaceAfter=No
6	,	,	PUNCT	_	_	5	punct	_	SpacesAfter=\s\s
7	jahtaa	Jahtaa	PROPN	_	Number=Sing	11	compound	_	_
8	oravia	oravia	CCONJ	_	_	11	compound	_	_
9	ja	ja	PROPN	_	_	11	compound	_	_
10	tsillailee	tsillailee	PROPN	_	Number=Sing	11	compound	_	_
11	kanssani	kanssani	PROPN	_	Number=Sing	2	parataxis	_	SpaceAfter=No
12	!	!	PUNCT	_	_	5	punct	_	SpacesAfter=\n


